#### Siste4mas de Recomendación 1

Vamos a ver sistemas de recomendacion en dos ejemplos:

1. Ejemplos básicos basandos en popularidad y Content Based
2. basado en filtro colaborativo y basando en modelos (SVD y KNN) aplicando el paquete Surprise

In [1]:
### Ojo surprise SVD funciona con una version de numpy<2 

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os

In [3]:
os.getcwd()

'C:\\Users\\tomas\\ML\\recom_sys'

#### Data set obtenido de kaggle

https://www.kaggle.com/datasets/zygmunt/goodbooks-10k

There have been good datasets for movies (Netflix, Movielens) and music (Million Songs) recommendation, but not for books. That is, until now.

This dataset contains ratings for ten thousand popular books. As to the source, let's say that these ratings were found on the internet. Generally, there are 100 reviews for each book, although some have less - fewer - ratings. Ratings go from one to five.

Both book IDs and user IDs are contiguous. For books, they are 1-10000, for users, 1-53424. All users have made at least two ratings. Median number of ratings per user is 8.

There are also books marked to read by the users, book metadata (author, year, etc.) and tags.

In [4]:
ratings_data = pd.read_csv('./data/ratings.csv')
books_metadata = pd.read_csv('./data/books.csv')
ratings_data.head(10)

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4
5,1,2077,4
6,1,2487,4
7,1,2900,5
8,1,3662,4
9,1,3922,5


In [5]:
books_metadata.head(10)

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...
5,6,11870085,11870085,16827462,226,525478817,9.780525e+12,John Green,2012.0,The Fault in Our Stars,...,2346404,2478609,140739,47994,92723,327550,698471,1311871,https://images.gr-assets.com/books/1360206420m...,https://images.gr-assets.com/books/1360206420s...
6,7,5907,5907,1540236,969,618260307,9.780618e+12,J.R.R. Tolkien,1937.0,The Hobbit or There and Back Again,...,2071616,2196809,37653,46023,76784,288649,665635,1119718,https://images.gr-assets.com/books/1372847500m...,https://images.gr-assets.com/books/1372847500s...
7,8,5107,5107,3036731,360,316769177,9.780317e+12,J.D. Salinger,1951.0,The Catcher in the Rye,...,2044241,2120637,44920,109383,185520,455042,661516,709176,https://images.gr-assets.com/books/1398034300m...,https://images.gr-assets.com/books/1398034300s...
8,9,960,960,3338963,311,1416524797,9.781417e+12,Dan Brown,2000.0,Angels & Demons,...,2001311,2078754,25112,77841,145740,458429,716569,680175,https://images.gr-assets.com/books/1303390735m...,https://images.gr-assets.com/books/1303390735s...
9,10,1885,1885,3060926,3455,679783261,9.780680e+12,Jane Austen,1813.0,Pride and Prejudice,...,2035490,2191465,49152,54700,86485,284852,609755,1155673,https://images.gr-assets.com/books/1320399351m...,https://images.gr-assets.com/books/1320399351s...


In [7]:
ratings_data.shape

(981756, 3)

In [11]:
ratings_data["book_id"].value_counts().head().to_frame()

,count
book_id,
1,100
5198,100
5215,100
5214,100
9803,100


In [15]:
rating_counts = pd.DataFrame(ratings_data["book_id"].value_counts())
rating_counts

,count
book_id,
1,100
5198,100
5215,100
5214,100
9803,100
...,...
9315,36
1935,34
9486,24


In [21]:
n_users = ratings_data.user_id.unique().shape[0]
n_items = ratings_data.book_id.unique().shape[0]
print(n_users, n_items)

53424 10000


In [82]:
df = ratings_data.merge(books_metadata, left_on="book_id", right_on="id", how="inner") 
# ojo , id del metadato es el book_id de los ratings
df

,book_id_x,user_id,rating,id,book_id_y,best_book_id,work_id,books_count,isbn,isbn13,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,314,5,1,2767052,2767052,2792775,272,439023483,9.780439e+12,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,1,439,3,1,2767052,2767052,2792775,272,439023483,9.780439e+12,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
2,1,588,5,1,2767052,2767052,2792775,272,439023483,9.780439e+12,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
3,1,1169,4,1,2767052,2767052,2792775,272,439023483,9.780439e+12,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
4,1,1185,4,1,2767052,2767052,2792775,272,439023483,9.780439e+12,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
981751,10000,48386,5,10000,8914,8914,11817,31,375700455,9.780376e+12,...,9162,9700,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...
981752,10000,49007,4,10000,8914,8914,11817,31,375700455,9.780376e+12,...,9162,9700,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...
981753,10000,49383,5,10000,8914,8914,11817,31,375700455,9.780376e+12,...,9162,9700,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...
981754,10000,50124,5,10000,8914,8914,11817,31,375700455,9.780376e+12,...,9162,9700,364,117,345,2031,4138,3069,https://images.gr-assets.com/books/1403194704m...,https://images.gr-assets.com/books/1403194704s...


In [84]:
n_users = df.user_id.unique().shape[0]
n_items = df.book_id_x.unique().shape[0]
print(n_users, n_items)

53424 10000


In [85]:
conteo_usuarios = df["user_id"].value_counts()
conteo_usuarios

user_id
30944    200
12874    200
12381    199
52036    199
28158    199
        ... 
10351      2
16592      2
24343      2
41314      2
27590      2
Name: count, Length: 53424, dtype: int64

In [86]:
conteo_books = df["book_id_x"].value_counts()
conteo_books

book_id_x
1       100
5198    100
5215    100
5214    100
9803    100
       ... 
9315     36
1935     34
9486     24
9345     11
7803      8
Name: count, Length: 10000, dtype: int64

#### 1. Popularity

Consideramos la siguiente metrica para ponderar los votos, no es lo mismo un 9 con 3 votos que un 8 con 100 votos.

WR = (v/(v+m))*R+(m/(v+m))*C

v is the number of votes for the book;  
m is the minimum votes required to be listed in the chart;  
R is the average rating of the book; And  
C is the mean vote across the whole report  
We already have v(vote_count) and R (vote_average  and C can be calculated as C= df['rating'].mean

Lo que hace esta formula es cuanto mas vvotos tiene mas pesa su propio rating. 
para libros con pocos votos, su rating pesa la mita y la media global de ratings la otra mitad.

In [80]:
C= df['rating'].mean()
C

3.8616453996813087

In [88]:
# Agrupar por USER_ID y calcular conteo + media
df_books = df.groupby("book_id_x")["rating"].agg(["count", "mean"]).reset_index()
df_books

,book_id_x,count,mean
0,1,100,4.240000
1,2,100,4.210000
2,3,100,3.090000
3,4,100,4.460000
4,5,100,3.890000
...,...,...,...
9995,9996,98,3.969388
9996,9997,89,4.426966
9997,9998,95,4.326316
9998,9999,99,3.727273


In [90]:
m= df_books['count'].quantile(0.9)
m

100.0

In [93]:
#WR = (v/(v+m))*R+(m/(v+m))*C
df_books['new_rating']=(df_books['count']/(df_books['count']+m))*df_books['mean']+((m/(df_books['count']+m))*C)
df_books

,book_id_x,count,mean,new_rating
0,1,100,4.240000,4.050823
1,2,100,4.210000,4.035823
2,3,100,3.090000,3.475823
3,4,100,4.460000,4.160823
4,5,100,3.890000,3.875823
...,...,...,...,...
9995,9996,98,3.969388,3.914972
9996,9997,89,4.426966,4.127855
9997,9998,95,4.326316,4.088023
9998,9999,99,3.727273,3.794797


In [115]:
df_books.sort_values(by="mean", ascending=False).head(10)

,book_id_x,count,mean,new_rating
7946,7947,89,4.820225,4.313040
6919,6920,100,4.780000,4.320823
5206,5207,100,4.780000,4.320823
9565,9566,99,4.777778,4.317410
8945,8946,93,4.774194,4.301371
6360,6361,100,4.770000,4.315823
3274,3275,100,4.770000,4.315823
5579,5580,100,4.750000,4.305823
6589,6590,100,4.750000,4.305823
4482,4483,100,4.750000,4.305823


Entonces, basandose en popularity, recomenasré a todos los usuarios los 5 libros con mayor valoracion y que no haya leido aun.

In [110]:
def get_recommendacion_pop(user_id, top):
    libros_leidos = df[(df["user_id"] == user_id)]["book_id_x"].to_frame()
    df_filtrado = df_books[~df_books["book_id_x"].isin(libros_leidos["book_id_x"])]
    df_top = df_filtrado.sort_values(by="mean", ascending=False).head(top)
    df_top2= df_top.merge(books_metadata, left_on="book_id_x", right_on="id", how="inner")
    return df_top2[["title", "id", "new_rating", "authors"]]

In [180]:
get_recommendacion_pop(1000, 10)

,title,id,new_rating,authors
0,ESV Study Bible,7947,4.313040,"Anonymous, Lane T. Dennis, Wayne A. Grudem"
1,The Indispensable Calvin and Hobbes,6920,4.320823,Bill Watterson
2,The Days Are Just Packed: A Calvin and Hobbes ...,5207,4.320823,Bill Watterson
3,Attack of the Deranged Mutant Killer Monster S...,9566,4.317410,Bill Watterson
4,The Divan,8946,4.301371,Hafez
5,There's Treasure Everywhere: A Calvin and Hobb...,6361,4.315823,Bill Watterson
6,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",3275,4.315823,"J.K. Rowling, Mary GrandPré"
7,It's a Magical World: A Calvin and Hobbes Coll...,4483,4.305823,Bill Watterson
8,The Authoritative Calvin and Hobbes: A Calvin ...,6590,4.305823,Bill Watterson
9,The Calvin and Hobbes Lazy Sunday Book,5580,4.305823,Bill Watterson


In [181]:
get_recommendacion_pop(1001, 10)

,title,id,new_rating,authors
0,ESV Study Bible,7947,4.313040,"Anonymous, Lane T. Dennis, Wayne A. Grudem"
1,The Indispensable Calvin and Hobbes,6920,4.320823,Bill Watterson
2,The Days Are Just Packed: A Calvin and Hobbes ...,5207,4.320823,Bill Watterson
3,Attack of the Deranged Mutant Killer Monster S...,9566,4.317410,Bill Watterson
4,The Divan,8946,4.301371,Hafez
5,"Harry Potter Boxed Set, Books 1-5 (Harry Potte...",3275,4.315823,"J.K. Rowling, Mary GrandPré"
6,There's Treasure Everywhere: A Calvin and Hobb...,6361,4.315823,Bill Watterson
7,It's a Magical World: A Calvin and Hobbes Coll...,4483,4.305823,Bill Watterson
8,The Calvin and Hobbes Lazy Sunday Book,5580,4.305823,Bill Watterson
9,The Authoritative Calvin and Hobbes: A Calvin ...,6590,4.305823,Bill Watterson


#### 2. Content Based

Se basa en buscar similitud entre diferentes libros, en base a palabrarclave, genero, etc.  
De tal manera que a los usuarios que les ha gustado el elemento A les genera recomendaciones parecidas a A de acierdo a palabras clave, genero, etc  
Aqui utilizo una tabla del data set que es book_tags donde cada palabra clave ya ha sido etiquetada con un valor numerico
luego hare simulitud del coseno en base a estas palabras clave y finalmente obtenenemos la recomendacion en base a los libros mas similares.

In [118]:
print(books_metadata.columns) 

Index(['id', 'book_id', 'best_book_id', 'work_id', 'books_count', 'isbn',
       'isbn13', 'authors', 'original_publication_year', 'original_title',
       'title', 'language_code', 'average_rating', 'ratings_count',
       'work_ratings_count', 'work_text_reviews_count', 'ratings_1',
       'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5', 'image_url',
       'small_image_url'],
      dtype='object')


In [120]:
book_tags = pd.read_csv('./data/book_tags.csv')
book_tags

,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716
...,...,...,...
999907,33288638,21303,7
999908,33288638,17271,7
999909,33288638,1126,7
999910,33288638,11478,7


In [135]:
tags = pd.read_csv('./data/tags.csv')
tags

,tag_id,tag_name
0,0,-
1,1,--1-
2,2,--10-
3,3,--12-
4,4,--122-
...,...,...
34247,34247,Ｃhildrens
34248,34248,Ｆａｖｏｒｉｔｅｓ
34249,34249,Ｍａｎｇａ
34250,34250,ＳＥＲＩＥＳ


In [122]:
book_tags2= book_tags.merge(books_metadata, left_on="goodreads_book_id", right_on="book_id", how="inner")[["id", "tag_id"]]
book_tags2

,id,tag_id
0,27,30574
1,27,11305
2,27,11557
3,27,8717
4,27,33114
...,...,...
999907,8892,21303
999908,8892,17271
999909,8892,1126
999910,8892,11478


Para hacerlo un poco mas visual me traido el valor de las tags, pero podría hacerse peerfectamente con los tag_id. la pivot table tarda unos minutos

In [136]:
book_tags3= book_tags2.merge(tags, left_on="tag_id", right_on="tag_id", how="inner")[["id", "tag_name"]]
book_tags3

,id,tag_name
0,27,to-read
1,27,fantasy
2,27,favorites
3,27,currently-reading
4,27,young-adult
...,...,...
999907,8892,neighbors
999908,8892,kindleunlimited
999909,8892,5-star-reads
999910,8892,fave-author


In [139]:
books_tags_df = book_tags3.pivot_table(index=["id"], columns=["tag_name"], aggfunc=lambda x: 1, fill_value=0)

In [140]:
books_tags_df 

tag_name,-,--1-,--10-,--12-,--122-,--166-,--17-,--19-,--2-,--258-,...,漫画,골든,﹏moonplus-reader﹏,ﺭﺿﻮﻯ-عاشور,ﻳﻮﺳﻒ-زيدان,Ｃhildrens,Ｆａｖｏｒｉｔｅｓ,Ｍａｎｇａ,ＳＥＲＩＥＳ,ｆａｖｏｕｒｉｔｅｓ
id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [126]:
from sklearn.metrics.pairwise import cosine_similarity

El calculo de la similarity lleva algunos minutos

In [141]:
cosine_sim = cosine_similarity(books_tags_df, books_tags_df)

In [142]:
cosine_sim

array([[1.        , 0.52261966, 0.53      , ..., 0.3       , 0.22      ,
        0.15      ],
       [0.52261966, 1.        , 0.48241815, ..., 0.3216121 , 0.18090681,
        0.16080605],
       [0.53      , 0.48241815, 1.        , ..., 0.21      , 0.13      ,
        0.16      ],
       ...,
       [0.3       , 0.3216121 , 0.21      , ..., 1.        , 0.17      ,
        0.24      ],
       [0.22      , 0.18090681, 0.13      , ..., 0.17      , 1.        ,
        0.22      ],
       [0.15      , 0.16080605, 0.16      , ..., 0.24      , 0.22      ,
        1.        ]])

#### algunas funciones
Son unas funciones para obtener informacion del libro. las usaremos despues en los modelos SVD 

In [143]:
import difflib
import random

def get_book_id(book_title, metadata):
    
    """
    Gets the book ID for a book title based on the closest match in the metadata dataframe.
    """
    
    existing_titles = list(metadata['title'].values)
    closest_titles = difflib.get_close_matches(book_title, existing_titles)
    book_id = metadata[metadata['title'] == closest_titles[0]]['id'].values[0]
    return book_id

def get_book_info(book_id, metadata):
    
    """
    Returns some basic information about a book given the book id and the metadata dataframe.
    """
    
    book_info = metadata[metadata['id'] == book_id][['id', 'isbn', 
                                                    'authors', 'title', 'original_title']]
    return book_info.to_dict(orient='records')


In [164]:
def recomendar_libro(titulo, df, sim_matrix):
    idx =  get_book_id(titulo, df)  # Obtener índice del libro
    similitudes = list(enumerate(sim_matrix[idx]))  # Obtener similitudes
    similares = sorted(similitudes, key=lambda x: x[1], reverse=True)[1:4]  # Ordenar por similitud y excluir el primero
    similares2 = pd.DataFrame(similares, columns=["id", "similaridad"])
    recomendados = [df.iloc[i[0]][["title",'authors']] for i in similares]  # Obtener títulos recomendados
    
    recomendados2 = df[df["id"].isin(similares2["id"])][["title", "id", "authors"]]
    return recomendados2 # pd.DataFrame(recomendados2)

In [182]:
# **Ejemplo**: Recomendar libros similares a "Libro A"
libros_recomendados = recomendar_libro("ESV Study Bible", books_metadata, cosine_sim)
print("Libros recomendados:", libros_recomendados['title'])
libros_recomendados

Libros recomendados: 2498                      Villette
2507                          Sula
4581    The Return of the Native  
Name: title, dtype: object


,title,id,authors
2498,Villette,2499,"Charlotte Brontë, A.S. Byatt, Μαρία Λαϊνά, Ign..."
2507,Sula,2508,Toni Morrison
4581,The Return of the Native,4582,"Thomas Hardy, Alexander Theroux"


In [ ]:
veamos las palabras clave asociadas a estos libros recomendados

In [183]:
book_tags3
z=book_tags3[book_tags3["id"].isin(libros_recomendados["id"])]

,id,tag_name
93800,2508,to-read
93801,2508,fiction
93802,2508,currently-reading
93803,2508,classics
93804,2508,favorites
...,...,...
205795,4582,my-collection
205796,4582,lit
205797,4582,tbr
205798,4582,book-group
